In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt

# font size for a publication
plt.rcParams.update({
    'font.size': 14,
    'axes.labelsize': 14,
    'axes.titlesize': 14,
})

## Summary Plot

In [ ]:
import pytplot

pytplot.del_data()

import download
import shockgeometry

reload(shockgeometry)

### Good Example

In [ ]:
DIR_FMT = shockgeometry.DIR_FMT
window = 3
deltat = np.timedelta64(90, "s")

dirname = "20161226_075900-20161226_080000"
analyzer = shockgeometry.AY76Analyzer(window, deltat)

tr = dirname.split("-")
t1 = pd.to_datetime(tr[0], format=DIR_FMT)
t2 = pd.to_datetime(tr[1], format=DIR_FMT)

download.load_hdf5(os.sep.join([dirname, "fast.h5"]), tplot=True)
download.load_hdf5(os.sep.join([dirname, "omni.h5"]), tplot=True)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    data_dict = shockgeometry.preprocess()
    result, figure = analyzer([t1, t2], data_dict[0], "test/" + dirname, 1)
    

In [ ]:
figure.subplots_adjust(left=0.15, right=0.85, top=0.92, bottom=0.07)
figure.suptitle(figure._suptitle.get_text(), fontsize=14)
axes = figure.get_axes()
for ax in axes:
    ax.xaxis.set_tick_params(labelsize=14)
    ax.yaxis.set_tick_params(labelsize=14)
    ax.xaxis.get_label().set_fontsize(14)
    ax.yaxis.get_label().set_fontsize(14)
figure

In [ ]:
# manual zoom
tt1 = pd.Timestamp("2016-12-26 07:57:00")
tt2 = pd.Timestamp("2016-12-26 08:02:00")

# change font size and some adjustments
figure.subplots_adjust(left=0.15, right=0.85, top=0.92, bottom=0.07)
figure.suptitle(figure._suptitle.get_text(), fontsize=14)
axes = figure.get_axes()
for ax in axes:
    ax.xaxis.set_tick_params(labelsize=14)
    ax.yaxis.set_tick_params(labelsize=14)
    ax.xaxis.get_label().set_fontsize(14)
    ax.yaxis.get_label().set_fontsize(14)
    ax.set_xlim([tt1, tt2])
figure.savefig("summary_good.pdf")


### Bad Example

In [ ]:
dirname = "20161209_112730-20161209_112900"
analyzer = shockgeometry.AY76Analyzer(window, deltat)

tr = dirname.split("-")
t1 = pd.to_datetime(tr[0], format=DIR_FMT)
t2 = pd.to_datetime(tr[1], format=DIR_FMT)

download.load_hdf5(os.sep.join([dirname, "fast.h5"]), tplot=True)
download.load_hdf5(os.sep.join([dirname, "omni.h5"]), tplot=True)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    data_dict = shockgeometry.preprocess()
    result, figure = analyzer([t1, t2], data_dict[0], "test/" + dirname, 1)


In [ ]:
# manual zoom
tt1 = pd.Timestamp("2016-12-09 11:26:00")
tt2 = pd.Timestamp("2016-12-09 11:31:00")

# change font size and some adjustments
figure.subplots_adjust(left=0.15, right=0.85, top=0.92, bottom=0.07)
figure.suptitle(figure._suptitle.get_text(), fontsize=14)
axes = figure.get_axes()
for ax in axes:
    ax.xaxis.set_tick_params(labelsize=14)
    ax.yaxis.set_tick_params(labelsize=14)
    ax.xaxis.get_label().set_fontsize(14)
    ax.yaxis.get_label().set_fontsize(14)
    ax.set_xlim([tt1, tt2])
figure.savefig("summary_bad.pdf")


## Definition for Shock Transition Region

In [ ]:
import wavestats
reload(wavestats)

In [ ]:
sc = 1
threshold = 0.10
suffix = "2048"
dirname = "20161226_075900-20161226_080000"

json_data = wavestats.read_json("test/" + dirname, sc)
t, x, y = wavestats.gather_transition_layer(json_data, sc=sc, threshold=threshold, suffix=suffix)
figure = wavestats.plot_timeseries(json_data, sc, suffix, t, x)

In [ ]:
# manual zoom
date = "2016-12-26"
tt1 = pd.Timestamp("{:s} 07:59:00".format(date))
tt2 = pd.Timestamp("{:s} 07:59:45".format(date))

# change font size and some adjustments
axes = figure.get_axes()
for ax in axes[0:3]:
    ax.xaxis.set_tick_params(labelsize=14)
    ax.yaxis.set_tick_params(labelsize=14)
    ax.xaxis.get_label().set_fontsize(14)
    ax.yaxis.get_label().set_fontsize(14)
    ax.set_xlim([tt1, tt2])
axes[2].set_rasterized(True)
figure.align_ylabels(axes[0:3])

figure.suptitle("MMS{:1d} Bow Shock {:s}".format(sc, str(tt1)), fontsize=14)
figure.savefig("power_timeseries.pdf")
figure

In [ ]:
import wavestats
reload(wavestats)

In [ ]:
sc = 1
threshold = 0.10
suffix = "2048"
dirname = "20161226_075900-20161226_080000"

json_data = wavestats.read_json("test/" + dirname, sc)
t, x, y = wavestats.gather_transition_layer(json_data, sc=sc, threshold=threshold, suffix=suffix)
figure = wavestats.plot_scatter(json_data, sc, suffix, x, y)

figure.suptitle("MMS{:1d} Bow Shock {:s}".format(sc, str(tt1)), fontsize=12)
figure.savefig("power_scatter.pdf")

## Event Summary

In [ ]:
# read CSV data for shock parameters and select quallity=1 events
sc = 1
suffix = 2048
prefix = "wavestats_{:d}_mms{:1d}".format(suffix, sc)
CSV_FILENAME = prefix + ".csv"
df = pd.read_csv(CSV_FILENAME, header=0, skiprows=0).drop_duplicates()
beta_median = df["Beta_omni_avg"].median()
condition = df["Beta_omni_avg"] < beta_median
df0 = df.copy()
df1 = df[condition]
df2 = df[~condition]

# number of events
Nevents = df0.shape[0]


def get_mach_number(df):
    ion_or_ele = "i"
    if ion_or_ele == "i":
        Ma_nif_avg = df["Ma_nif_i_avg"]
        Ma_nif_err = df["Ma_nif_i_err"]
    elif ion_or_ele == "e":
        Ma_nif_avg = df["Ma_nif_e_avg"]
        Ma_nif_err = df["Ma_nif_e_err"]
    else:
        raise ValueError("ion_or_ele must be either 'i' or 'e'")
    return Ma_nif_avg, Ma_nif_err


def get_cos_tbn(df):
    cos_tbn_avg = df["cos_tbn_avg"]
    cos_tbn_err = df["cos_tbn_err"]
    return cos_tbn_avg, cos_tbn_err


def get_beta(df):
    beta_avg = df["Beta_omni_avg"]
    beta_err = df["Beta_omni_err"]
    return beta_avg, beta_err


def get_beta_e(df):
    beta_e_avg = df["Beta_e_avg"]
    beta_e_err = df["Beta_e_err"]
    return beta_e_avg, beta_e_err


def get_beta_i(df):
    beta_i_avg = df["Beta_i_avg"]
    beta_i_err = df["Beta_i_err"]
    return beta_i_avg, beta_i_err


def get_wavepower(df):
    # store wave power
    wavepow_med = [0] * 3
    wavepow_err = [0] * 3
    for i in range(3):
        key = "wavepow{:1d}".format(i)
        wavepow_med[i] = np.array(df[key + "_med"])
        wavepow_err[i] = np.zeros((2, wavepow_med[i].size))
        wavepow_err[i][0, :] = np.array(df[key + "_min"])
        wavepow_err[i][1, :] = np.array(df[key + "_max"])
    return wavepow_med, wavepow_err

In [ ]:
# event selection
df = df0
Nevents = df.shape[0]
Ma_nif_avg, Ma_nif_err = get_mach_number(df)
cos_tbn_avg, cos_tbn_err = get_cos_tbn(df)
beta_avg, beta_err = get_beta(df)
beta_e_avg, beta_e_err = get_beta_e(df)
beta_i_avg, beta_i_err = get_beta_i(df)

cost = np.linspace(-1.0, +1.0, 51)
Ma0 = 0.5 * np.sqrt(1836)
plt.plot(cost, 1 * Ma0 * np.abs(cost), "--", color="gray", lw=1)
plt.plot(cost, 2 * Ma0 * np.abs(cost), "-.", color="gray", lw=1)
plt.plot(cos_tbn_avg, Ma_nif_avg, "o", ms=2, color="k")
plt.errorbar(
    cos_tbn_avg,
    Ma_nif_avg,
    xerr=cos_tbn_err,
    yerr=Ma_nif_err,
    fmt="none",
    lw=2,
    color="k",
    alpha=0.25,
)

plt.gca().xaxis.set_major_locator(mpl.ticker.MultipleLocator(0.20))
plt.gca().xaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.05))
plt.gca().yaxis.set_major_locator(mpl.ticker.MultipleLocator(5.0))
plt.gca().yaxis.set_minor_locator(mpl.ticker.MultipleLocator(1.0))
plt.xlim(-1, +1)
plt.ylim(+1, +25)
plt.grid()
plt.xlabel(r"$\cos \theta_{Bn}$")
plt.ylabel(r"$M_A$")

plt.title("MMS{:1d} ({:d} Events)".format(sc, Nevents))
plt.tight_layout()

plt.savefig("event_summary.pdf")

In [ ]:
# relation between power of different frequencies
kwargs = [
    {"fmt": "none", "lw": 1, "ms": 2, "color": "g", "alpha": 0.25},
    {"fmt": "none", "lw": 1, "ms": 2, "color": "b", "alpha": 0.25},
]

wavepow_med, wavepow_err = get_wavepower(df0)

label1 = r"$0.10 \leq f/f_{\rm ce} \leq 1.0$"
label2 = r"$0.20 \leq f/f_{\rm ce} \leq 1.0$"
plt.plot(wavepow_med[0], wavepow_med[1], "o", ms=2, color="g", label=label1)
plt.plot(wavepow_med[0], wavepow_med[2], "o", ms=2, color="b", label=label2)
plt.errorbar(wavepow_med[0], wavepow_med[1], xerr=wavepow_err[0], yerr=wavepow_err[1], **kwargs[0])
plt.errorbar(wavepow_med[0], wavepow_med[2], xerr=wavepow_err[0], yerr=wavepow_err[2], **kwargs[1])

plt.legend(loc="upper left")
plt.loglog()
plt.xlim(3e-5, 3e-1)
plt.ylim(1e-7, 1e-1)
plt.grid()
plt.xlabel(r"Integrated Power ($0.05 \leq f/f_{\rm ce} \leq 1.0$) [$B_0^2$]")
plt.ylabel(r"Integrated Power [$B_0^2$]")

Nevents = len(df1) + len(df2)
plt.title("MMS{:1d} ({:d} Events)".format(sc, Nevents))
plt.loglog()
plt.tight_layout()

plt.savefig('power_summary.pdf')

# Theoretical Threshold

In [ ]:
# show resonant velocity and energy
from scipy import constants


def omega(k):
    mie = 1836
    A = 1 + (1 + 1 / mie) / k**2
    B = 1 / mie - 1
    C = -1 / mie
    return (-B + np.sqrt(B**2 - 4 * A * C)) / (2 * A)


k = np.geomspace(1e-2, 1e1, 51)
w = omega(k)
v = (1 - w) / k

fig, axes = plt.subplots(2, 1, figsize=(6, 8))
fig.subplots_adjust(hspace=0.3)

# normalized resonant velocity
axes[0].plot(w, v, "k-")
axes[0].set_xlim(1e-2, 1e0)
axes[0].set_ylim(1e-2, 1e1)
axes[0].set_xlabel(r"$\omega/\omega_{\rm ce}$")
axes[0].set_ylabel(r"$v_{\rm res}/V_{\rm A,e}$")
axes[0].loglog()
axes[0].grid(which="both")

# energy in eV for particuler B and n
n = 1.0e1
B = 1.0e1
label = "$B =$ {:4.1f} [nT]; $n = $ {:4.1f} [cm$^{{-3}}$]".format(B, n)
Vae = 21.8 * B / np.sqrt(n) * np.sqrt(1836) * 1e3
E = 0.5 * constants.m_e / constants.e * (v * Vae) ** 2

axes[1].plot(w, E, "k-")
axes[1].set_xlim(1e-2, 1e0)
axes[1].set_ylim(1e-0, 1e4)
axes[1].set_xlabel(r"$\omega/\omega_{\rm ce}$")
axes[1].set_ylabel(r"$E_{\rm res}$ [eV]")
axes[1].set_title(label)
axes[1].loglog()
axes[1].grid(which="both")

In [ ]:
from scipy import integrate
from scipy import optimize


def omega(k):
    mie = 1836
    A = 1 + (1 + 1 / mie) / k**2
    B = 1 / mie - 1
    C = -1 / mie
    return (-B + np.sqrt(B**2 - 4 * A * C)) / (2 * A)


def integrand(k):
    mie = 1836
    A = 1 + (1 + 1 / mie) / k**2
    B = 1 / mie - 1
    C = -1 / mie
    w = (-B + np.sqrt(B**2 - 4 * A * C)) / (2 * A)
    v = (1 - w) / k
    return v**3


def energy_integral(kmin, kmax):
    val, _ = integrate.quad(integrand, kmin, kmax)
    return val


# integration range
fmax = np.array([1] * 3) * 9.999e-1
fmin = np.array([0.5e-1, 1.0e-1, 2.0e-1])
kmax = np.zeros(3)
kmin = np.zeros(3)
Eint = np.zeros(3)

print("*** Energy Integral ***")
for i in range(3):
    kmax[i] = optimize.root_scalar(
        lambda x: omega(x) - fmax[i], bracket=[1.0e-2, 1.0e2], method="brentq"
    ).root
    kmin[i] = optimize.root_scalar(
        lambda x: omega(x) - fmin[i], bracket=[1.0e-2, 1.0e2], method="brentq"
    ).root
    Eint[i] = energy_integral(kmin[i], kmax[i])
    print("- {:4.2f} < f/fce < {:4.2f} : {:12.5e}".format(fmin[i], fmax[i], Eint[i]))

In [ ]:
def create_figure_axes():
    fig, axs = plt.subplots(1, 3, figsize=(14.4, 4.8))
    plt.subplots_adjust(top=0.89, bottom=0.13, left=0.06, right=0.98, wspace=0.22, hspace=0.25)
    return fig, axs

def plot_integrated_power(axs, xval, xerr, yval, yerr, **kw):
    # 0.05 <= f/fce <= 1.00
    plt.sca(axs[0])
    plt.errorbar(xval, yval[0], xerr=xerr, yerr=yerr[0], **kw)
    plt.title(r"$0.05 \leq f/f_{\rm ce} \leq 1.0$")
    # 0.10 <= f/fce <= 1.00
    plt.sca(axs[1])
    plt.errorbar(xval, yval[1], xerr=xerr, yerr=yerr[1], **kw)
    plt.title(r"$0.10 \leq f/f_{\rm ce} \leq 1.0$")
    # 0.20 <= f/fce <= 1.00
    plt.sca(axs[2])
    plt.errorbar(xval, yval[2], xerr=xerr, yerr=yerr[2], **kw)
    plt.title(r"$0.20 \leq f/f_{\rm ce} \leq 1.0$")

    for ax in axs:
        ax.set_yscale("log")
        ax.set_ylim(1e-7, 1e0)
        ax.yaxis.set_major_locator(mpl.ticker.LogLocator(numticks=10))
        ax.yaxis.set_minor_locator(
            mpl.ticker.LogLocator(numticks=10, subs=np.arange(1.0, 10.0) * 0.1)
        )
        ax.grid()
    axs[0].set_ylabel(r"Integrated Power [$B_0^2$]")


def event_selection(df, theta_min, theta_max):
    condition1 = np.abs(df["cos_tbn_avg"]) > np.cos(np.deg2rad(theta_max))
    condition2 = np.abs(df["cos_tbn_avg"]) < np.cos(np.deg2rad(theta_min))
    return df[np.logical_and(condition1, condition2)]


def get_wavepower(df):
    # store wave power
    wavepow_med = [0] * 3
    wavepow_err = [0] * 3
    for i in range(3):
        key = "wavepow{:1d}".format(i)
        wavepow_med[i] = np.array(df[key + "_med"])
        wavepow_err[i] = np.zeros((2, wavepow_med[i].size))
        wavepow_err[i][0, :] = np.array(df[key + "_min"])
        wavepow_err[i][1, :] = np.array(df[key + "_max"])
    return wavepow_med, wavepow_err

def get_title(num_events, beta_median, theta_min=None, theta_max=None):
    events = "{:d} events".format(num_events)
    beta = r"${{\rm median}}(\beta_{{\rm omni}}) = {:5.2f}$".format(beta_median)
    if theta_min is not None and theta_max is not None:
        theta = r"$|\cos {:d}^\circ| < |\cos \theta_{{Bn}}| < |\cos {:d}^\circ|$".format(
            theta_max, theta_min
        )
        title = r"MMS{:1d} ({:s}, {:s}, {:s})".format(sc, events, theta, beta)
    else:
        title = r"MMS{:1d} ({:s}, {:s})".format(sc, events, beta)
    return title

#
# divide events into two groups by beta
#
beta_median = df0["Beta_omni_avg"].median()
condition = df0["Beta_omni_avg"] < beta_median
df1 = df0[condition]
df2 = df0[~condition]

save_figure = True

In [ ]:
dfs = [df1, df2]

kwargs = [
    {"fmt": "b+", "ms": 3, "ecolor": (0.0, 0.0, 1.0, 0.2)},
    {"fmt": "r+", "ms": 3, "ecolor": (1.0, 0.0, 0.0, 0.2)},
]

fig, axs = create_figure_axes()

for i in range(2):
    xval, xerr = get_mach_number(dfs[i])
    yval, yerr = get_wavepower(dfs[i])
    plot_integrated_power(axs, xval, xerr, yval, yerr, **kwargs[i])

# xaxis
for ax in axs:
    ax.set_xlabel(r"$M_A$")
    ax.set_xlim(0.0, 25.0)
    ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(5.0))
    ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(1.0))
    ax.grid()

# title
num_events = len(dfs[0]) + len(dfs[1])
plt.suptitle(get_title(num_events, beta_median), fontsize=14)
plt.show()

if save_figure:
    fig.savefig(prefix + "_mach_nif_all.png".format(sc), dpi=300)
    fig.savefig(prefix + "_mach_nif_all.pdf".format(sc), dpi=300)

In [ ]:
theta_min = 60
theta_max = 80
dfs = [df1, df2]
for i in range(2):
    dfs[i] = event_selection(dfs[i], theta_min, theta_max)

kwargs = [
    {"fmt": "b+", "ms": 3, "ecolor": (0.0, 0.0, 1.0, 0.2)},
    {"fmt": "r+", "ms": 3, "ecolor": (1.0, 0.0, 0.0, 0.2)},
]

fig, axs = create_figure_axes()

for i in range(2):
    xval, xerr = get_mach_number(dfs[i])
    yval, yerr = get_wavepower(dfs[i])
    plot_integrated_power(axs, xval, xerr, yval, yerr, **kwargs[i])

# xaxis
for ax in axs:
    ax.set_xlabel(r"$M_A$")
    ax.set_xlim(0.0, 25.0)
    ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(5.0))
    ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(1.0))
    ax.grid()

# title
num_events = len(dfs[0]) + len(dfs[1])
plt.suptitle(get_title(num_events, beta_median, theta_min, theta_max), fontsize=14)
plt.show()

if save_figure:
    fig.savefig(prefix + "_mach_nif_selected.png".format(sc), dpi=300)
    fig.savefig(prefix + "_mach_nif_selected.pdf".format(sc), dpi=300)

In [ ]:
dfs = [df1, df2]

kwargs = [
    {"fmt": "b+", "ms": 3, "ecolor": (0.0, 0.0, 1.0, 0.2)},
    {"fmt": "r+", "ms": 3, "ecolor": (1.0, 0.0, 0.0, 0.2)},
]

fig, axs = create_figure_axes()

for i in range(2):
    Ma_nif_avg, Ma_nif_err = get_mach_number(dfs[i])
    cos_tbn_avg, cos_tbn_err = get_cos_tbn(dfs[i])
    xval = Ma_nif_avg / np.abs(cos_tbn_avg)
    xerr = np.sqrt((Ma_nif_err**2 + cos_tbn_err**2 * xval**2) / cos_tbn_avg**2)
    yval, yerr = get_wavepower(dfs[i])
    plot_integrated_power(axs, xval, xerr, yval, yerr, **kwargs[i])

# plot theoretical threshold
eta = 0.5
Bstl = 2.0
Ma_htf = np.linspace(1.0e0, 1.0e3, 51)
for i in range(3):
    plt.sca(axs[i])
    threshold = 2 / (3 * np.pi * eta) * Bstl * Eint[i] / Ma_htf**2
    plt.fill_between(Ma_htf, threshold/3.0, threshold*3.0, color="k", alpha=0.1)

# xaxis
for ax in axs:
    ax.set_xlabel(r"$M_A / \cos \theta_{Bn}$")
    ax.set_xlim(4.0e0, 1.0e3)
    ax.xaxis.set_major_locator(mpl.ticker.LogLocator(numticks=10))
    ax.xaxis.set_minor_locator(mpl.ticker.LogLocator(numticks=10, subs=np.arange(1.0, 10.0) * 0.1))
    ax.grid()
    ax.loglog()

# title
num_events = len(dfs[0]) + len(dfs[1])
plt.suptitle(get_title(num_events, beta_median), fontsize=14)
plt.show()

if save_figure:
    fig.savefig(prefix + "_mach_htf_all.png".format(sc), dpi=300)
    fig.savefig(prefix + "_mach_htf_all.pdf".format(sc), dpi=300)

In [ ]:
theta_min = 60
theta_max = 80
dfs = [df1, df2]
for i in range(2):
    dfs[i] = event_selection(dfs[i], theta_min, theta_max)

kwargs = [
    {"fmt": "b+", "ms": 3, "ecolor": (0.0, 0.0, 1.0, 0.2)},
    {"fmt": "r+", "ms": 3, "ecolor": (1.0, 0.0, 0.0, 0.2)},
]

fig, axs = create_figure_axes()

for i in range(2):
    Ma_nif_avg, Ma_nif_err = get_mach_number(dfs[i])
    cos_tbn_avg, cos_tbn_err = get_cos_tbn(dfs[i])
    xval = Ma_nif_avg / np.abs(cos_tbn_avg)
    xerr = np.sqrt((Ma_nif_err**2 + cos_tbn_err**2 * xval**2) / cos_tbn_avg**2)
    yval, yerr = get_wavepower(dfs[i])
    plot_integrated_power(axs, xval, xerr, yval, yerr, **kwargs[i])

# plot theoretical threshold
eta = 0.5
Bstl = 2.0
Ma_htf = np.linspace(1.0e0, 1.0e3, 51)
for i in range(3):
    plt.sca(axs[i])
    threshold = 2 / (3 * np.pi * eta) * Bstl * Eint[i] / Ma_htf**2
    plt.fill_between(Ma_htf, threshold/3.0, threshold*3.0, color="k", alpha=0.1)

# xaxis
for ax in axs:
    ax.set_xlabel(r"$M_A / \cos \theta_{Bn}$")
    ax.set_xlim(4.0e0, 1.0e3)
    ax.xaxis.set_major_locator(mpl.ticker.LogLocator(numticks=10))
    ax.xaxis.set_minor_locator(mpl.ticker.LogLocator(numticks=10, subs=np.arange(1.0, 10.0) * 0.1))
    ax.grid()
    ax.loglog()

# title
num_events = len(dfs[0]) + len(dfs[1])
plt.suptitle(get_title(num_events, beta_median, theta_min, theta_max), fontsize=14)
plt.show()

if save_figure:
    fig.savefig(prefix + "_mach_htf_selected.png".format(sc), dpi=300)
    fig.savefig(prefix + "_mach_htf_selected.pdf".format(sc), dpi=300)

In [ ]:
dfs = [df1, df2]

chi = 0.5
kwargs = [
    {"fmt": "b+", "ms": 3, "ecolor": (0.0, 0.0, 1.0, 0.2)},
    {"fmt": "r+", "ms": 3, "ecolor": (1.0, 0.0, 0.0, 0.2)},
]

fig, axs = create_figure_axes()

for i in range(2):
    Ma_nif_avg, Ma_nif_err = get_mach_number(dfs[i])
    cos_tbn_avg, cos_tbn_err = get_cos_tbn(dfs[i])
    beta_e_avg, beta_e_err = get_beta_e(dfs[i])
    xval = Ma_nif_avg / np.abs(cos_tbn_avg) * beta_e_avg**chi
    xerr = xval * np.sqrt(
        (Ma_nif_err / Ma_nif_avg) ** 2
        + (cos_tbn_err / cos_tbn_avg) ** 2
        + (chi * beta_e_err / beta_e_avg) ** 2
    )
    yval, yerr = get_wavepower(dfs[i])
    plot_integrated_power(axs, xval, xerr, yval, yerr, **kwargs[i])

# xaxis
for ax in axs:
    ax.set_xlabel(r"$\beta_e^{1/2} M_A / \cos \theta_{Bn}$")
    ax.set_xlim(4.0e0, 1.0e3)
    ax.xaxis.set_major_locator(mpl.ticker.LogLocator(numticks=10))
    ax.xaxis.set_minor_locator(mpl.ticker.LogLocator(numticks=10, subs=np.arange(1.0, 10.0) * 0.1))
    ax.grid()
    ax.loglog()

# title
num_events = len(dfs[0]) + len(dfs[1])
plt.suptitle(get_title(num_events, beta_median), fontsize=14)
plt.show()

if save_figure:
    fig.savefig(prefix + "_mach_beta_all.png".format(sc), dpi=300)
    fig.savefig(prefix + "_mach_beta_all.pdf".format(sc), dpi=300)

In [ ]:
theta_min = 60
theta_max = 80
dfs = [df1, df2]
for i in range(2):
    dfs[i] = event_selection(dfs[i], theta_min, theta_max)

chi = 0.5
kwargs = [
    {"fmt": "b+", "ms": 3, "ecolor": (0.0, 0.0, 1.0, 0.2)},
    {"fmt": "r+", "ms": 3, "ecolor": (1.0, 0.0, 0.0, 0.2)},
]

fig, axs = create_figure_axes()

for i in range(2):
    Ma_nif_avg, Ma_nif_err = get_mach_number(dfs[i])
    cos_tbn_avg, cos_tbn_err = get_cos_tbn(dfs[i])
    beta_e_avg, beta_e_err = get_beta_e(dfs[i])
    xval = Ma_nif_avg / np.abs(cos_tbn_avg) * beta_e_avg**chi
    xerr = xval * np.sqrt(
        (Ma_nif_err / Ma_nif_avg) ** 2
        + (cos_tbn_err / cos_tbn_avg) ** 2
        + (chi * beta_e_err / beta_e_avg) ** 2
    )
    yval, yerr = get_wavepower(dfs[i])
    plot_integrated_power(axs, xval, xerr, yval, yerr, **kwargs[i])

# xaxis
for ax in axs:
    ax.set_xlabel(r"$\beta_e^{1/2} M_A / \cos \theta_{Bn}$")
    ax.set_xlim(4.0e0, 1.0e3)
    ax.xaxis.set_major_locator(mpl.ticker.LogLocator(numticks=10))
    ax.xaxis.set_minor_locator(mpl.ticker.LogLocator(numticks=10, subs=np.arange(1.0, 10.0) * 0.1))
    ax.grid()
    ax.loglog()

# title
num_events = len(dfs[0]) + len(dfs[1])
plt.suptitle(get_title(num_events, beta_median, theta_min, theta_max), fontsize=14)
plt.show()

if save_figure:
    fig.savefig(prefix + "_mach_beta_selected.png".format(sc), dpi=300)
    fig.savefig(prefix + "_mach_beta_selected.pdf".format(sc), dpi=300)